In [5]:
import numpy as np
import pandas as pd
import config # a python file that contains path to TIMSS data files
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn import metrics
#from sklearn import tree
#from sklearn import BaggingClassifier
#from sklearn import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ml_data_1 = pd.read_csv(config.ml_data_1, index_col=False)

In [6]:
x = ml_data_1.drop(columns='mean_PV')
x.sample()

,BSBG05D,BSBG04,BSBG13E,BSBG13D,BSBG14A,BSBM16A,BSBM17D,BSBM18C,BSBM20E,BSBM20I
2022,No,Enough to fill two bookcases (101–200 books),Agree a lot,Agree a lot,At least once a week,Agree a little,Agree a lot,Every or almost every lesson,Agree a lot,Agree a lot


In [7]:
y = ml_data_1.mean_PV
y.sample()

625    503.206364
Name: mean_PV, dtype: float64

In [8]:
from kmodes.kmodes import KModes

# Instantiate the k-modes model
km = KModes(n_clusters=5, init='Huang', n_init=4, verbose=2, n_jobs=2)

# Fit the model to your categorical data
clusters = km.fit_predict(x)


Init: initializing centroids
Init: initializing centroids
Init: initializing clusters
Init: initializing clusters
Starting iterations...
Starting iterations...
Run 1, iteration: 1/100, moves: 509, cost: 33186.0
Run 2, iteration: 1/100, moves: 3136, cost: 30471.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 2/100, moves: 948, cost: 32324.0
Run 3, iteration: 1/100, moves: 2416, cost: 30864.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 3/100, moves: 752, cost: 32324.0
Run 4, iteration: 1/100, moves: 2868, cost: 30215.0
Run 4, iteration: 2/100, moves: 1705, cost: 30215.0
Best run was number 4


In [9]:
clusters.max()

4

In [11]:
clusters

array([2, 2, 2, ..., 2, 0, 0], dtype=uint16)

Now that we have our clusters, we will add their numbers to each data point in our labels data in order to separate each cluster alone and build a separate model for each:

In [12]:
x['clusters'] = clusters

In [14]:
x.describe()

,clusters
count,8458.000000
mean,1.386616
std,1.373483
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,4.000000


In [30]:
x.sample(5)

,BSBG05D,BSBG04,BSBG13E,BSBG13D,BSBG14A,BSBM16A,BSBM17D,BSBM18C,BSBM20E,BSBM20I,clusters
5080,Yes,None or very few (0–10 books),Agree a lot,Agree a lot,Never,Disagree a little,Agree a lot,About half the lessons,Agree a little,Agree a lot,2
5706,No,None or very few (0–10 books),Agree a lot,Agree a little,A few times a year,Agree a lot,Agree a lot,About half the lessons,Agree a lot,Agree a lot,3
7975,Yes,Enough to fill one bookcase (26–100 books),Agree a little,Agree a lot,Never,Agree a lot,Agree a lot,Never,Agree a lot,Agree a lot,0
3533,No,None or very few (0–10 books),Agree a lot,Agree a lot,Never,Agree a lot,Agree a lot,Some lessons,Agree a lot,Agree a lot,2
7779,Yes,Enough to fill one bookcase (26–100 books),Agree a lot,Agree a lot,At least once a week,Agree a lot,Agree a lot,Every or almost every lesson,Agree a lot,Agree a little,0


Let's see what clusters do we have:

In [16]:
list_of_clusters = x['clusters'].unique()
list_of_clusters

array([2, 3, 0, 4, 1], dtype=uint16)

Let's put each cluster in a different dataframe:

In [24]:
x_cluster_0 = x.query('clusters==0')
x_cluster_1 = x.query('clusters==1')
x_cluster_2 = x.query('clusters==2')
x_cluster_3 = x.query('clusters==3')
x_cluster_4 = x.query('clusters==4')

In [29]:
x_cluster_4.count()

BSBG05D     1025
BSBG04      1025
BSBG13E     1025
BSBG13D     1025
BSBG14A     1025
BSBM16A     1025
BSBM17D     1025
BSBM18C     1025
BSBM20E     1025
BSBM20I     1025
clusters    1025
dtype: int64